# Global Sertlik Matrisi ve Kuvvet Vektörünün Oluşturulması

Geçtiğimiz bölümde eleman sertlik matrisini ve kuvvet vektörünü referans elemanlar için integral formunda türetmiştik. Şimdi bu türettiğimiz denklemleri nümerik olarak hesaplayıp, global sertlik matrisini ve kuvvet vektörünü elde edelim. Hesaplamalara başlamadan önce problemin çözümünde kullanacağımız ağı (mesh) tanımlayalım. Şekil 3'te kullanacağımız ağ yapısı gösterilmiştir. Gri noktalar düğüm noktalarını göstermektedir. Eleman numaraları elemanların içinde yuvarlak içerisinde gösterilirken, düğüm noktalarının yanında numaraları ve $x-y$ koordinat sisteminde konumları metre cinsinden gösterilmektedir. 

<img src="Resimler/Ders8/Sekil3.png" align="center" width= "700"/>

## Global Sertlik Matrisinin Oluşturulması

Öncelikle 8b'de referans eleman için türettiğimiz yerel sertlik matrisini ifade eden integrali hatırlayalım:

$$\int_{-1}^{1}\int_{-1}^{1} \mathbf{N}^{T}\mathbf{J}^{T}\mathbf{H}^{T}\mathbf{C}\mathbf{H}\mathbf{J}\mathbf{N}d\eta d\xi det(\mathbf{J}).\tag{1} $$


Şimdi de integral içerisindeki matrisleri hatırlayalım:

$$ \mathbf{C}= \frac{E}{(1-\nu^{2})} \begin{bmatrix} 1 & \nu & 0 \\ \nu & 1 & 0 \\ 0 & 0 & \frac{1-\nu}{2} \end{bmatrix}, $$
$$ \mathbf{H}=\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 1 & 1 & 0 \end{bmatrix},$$
$$\mathbf{J}=\begin{bmatrix} \mathbf{J^{-1}} & 0 \\ 0 & \mathbf{J^{-1}} \end{bmatrix},$$
$$\mathbf{N}=\begin{bmatrix} \frac{\partial N_1}{\partial \xi}  & 0 & \frac{\partial N_2}{\partial \xi}  & 0 & \frac{\partial N_3}{\partial \xi} & 0 & \frac{\partial N_4}{\partial \xi} & 0 & \\ \\ \frac{\partial N_1}{\partial \eta} & 0  & \frac{\partial N_2}{\partial \eta} & 0  & \frac{\partial N_3}{\partial \eta} & 0 & \frac{\partial N_4}{\partial \eta} & 0 \\ \\ 0  & \frac{\partial N_1}{\partial \xi}  & 0 & \frac{\partial N_2}{\partial \xi} & 0  & \frac{\partial N_3}{\partial \xi}  & 0 & \frac{\partial N_4}{\partial \xi}\\ \\ 0  & \frac{\partial N_1}{\partial \eta}  & 0 & \frac{\partial N_2}{\partial \eta} & 0  & \frac{\partial N_3}{\partial \eta}  & 0 & \frac{\partial N_4}{\partial \eta} \end{bmatrix}.$$

Görülebileceği üzere $\mathbf{C}$ ve $\mathbf{H}$ sabit değerli matrislerdir. İntegrali hesaplayabilmek için $\mathbf{J}$ ve $\mathbf{N}$ matirslerini tanımlamamız gerekir. Öncelikle bunun için 7. bölümde olduğu gibi *sympy* kütüphanesini kullanalım. 

In [57]:
import sys

$\mathbf{N}$ matrisini yazabilmek için şekil fonksiyonlarını sembolik olarak yazalım:

In [58]:
import sympy as sp
import numpy as np

xi = sp.Symbol("xi")
eta = sp.Symbol("eta")
x_ = sp.symbols("x1,x2,x3,x4")
y_ = sp.symbols("y1,y2,y3,y4")
şekil = sp.Matrix([(1+xi)*(1+eta)/4,
                   (1-xi)*(1+eta)/4,
                   (1-xi)*(1-eta)/4,
                   (1+xi)*(1-eta)/4])
şekil

Matrix([
[(eta + 1)*(xi + 1)/4],
[(1 - xi)*(eta + 1)/4],
[(1 - eta)*(1 - xi)/4],
[(1 - eta)*(xi + 1)/4]])

Şimdi şekil fonksiyonlarının $\xi$ ve $\eta$'e göre türevlerini alıp $\mathbf{N}$'i tanımlayalım:

In [59]:
gradyen_xi = sp.diff(şekil, xi)
gradyen_eta = sp.diff(şekil, eta)

N = sp.Matrix([[gradyen_xi[0],0,gradyen_xi[1],0,gradyen_xi[2],0, gradyen_xi[3],0],[gradyen_eta[0],0,gradyen_eta[1],0,gradyen_eta[2],0, gradyen_eta[3],0],[0,gradyen_xi[0],0,gradyen_xi[1],0, gradyen_xi[2],0,gradyen_xi[3]],[0,gradyen_eta[0],0,gradyen_eta[1],0, gradyen_eta[2],0,gradyen_eta[3]]])

N

Matrix([
[eta/4 + 1/4,           0, -eta/4 - 1/4,            0, eta/4 - 1/4,           0, 1/4 - eta/4,           0],
[ xi/4 + 1/4,           0,   1/4 - xi/4,            0,  xi/4 - 1/4,           0, -xi/4 - 1/4,           0],
[          0, eta/4 + 1/4,            0, -eta/4 - 1/4,           0, eta/4 - 1/4,           0, 1/4 - eta/4],
[          0,  xi/4 + 1/4,            0,   1/4 - xi/4,           0,  xi/4 - 1/4,           0, -xi/4 - 1/4]])

Şimdi jakopyen matrisini oluşturup 1. eleman için $\mathbf{J}$ matrisini hesaplayalım:

In [60]:
c_şekil = sp.Matrix([0, 0])
for i in range(4):
    c_şekil[0] += x_[i] * şekil[i]
    c_şekil[1] += y_[i] * şekil[i]
j = c_şekil.jacobian([xi, eta]).T

# x_koord = [0,     0.16,    0.16,  0.0] 
# y_koord = [-0.32,-0.20,   -0.10, -0.16]

x_koord = [0.16,     0.13,    0.0,  0.0] 
y_koord = [-0.10,   -0.0,   -0.0, -0.16]
    
for nokta in range(4):
    j = j.subs(x_[nokta],x_koord[nokta]) 
    j = j.subs(y_[nokta],y_koord[nokta])

J= sp.Matrix.zeros(4,4)
J[:2,:2] = j.inv()
J[2:,2:] = j.inv()
det = j.det()
det

-0.000975*eta + 0.0006*xi + 0.004825

Son olarak $\mathbf{C}$ ve $\mathbf{H}$ matrislerini tanımlayıp, 1. eleman için eleman sertlik matrisini elde edelim:

In [61]:
E=1E7
v=0.3
H=sp.Matrix([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
C=sp.Matrix([[1,v,0],[v,1,0],[0,0,(1-v)/2]])*E/(1-v**2)
C

Matrix([
[10989010.989011, 3296703.2967033,                0],
[3296703.2967033, 10989010.989011,                0],
[              0,               0, 3846153.84615385]])

In [62]:
K_ele_sys=N.T*J.T*H.T*C*H*J*N*det
from scipy import integrate
from sympy.utilities import lambdify

K_e = np.zeros((8,8))
for i in range(8):
    for j in range(8):
        # scipy'da bulunan integrasyon fonksiyonunu kullanmak için
        # hücre değerlerini geçici olarak 'lambda' formatına çevirmemiz gerekiyor.
        geçici_fonksiyon = lambdify( (xi,eta), K_ele_sys[i,j], 'math' )
        K_e[i,j] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]
K_e


array([[ 5907470.124, -2001814.091,  -508264.159,   540694.265, -3269537.137,  1858337.667, -2129668.827,  -397217.842],
       [-2001814.091,  5242029.212,   815419.54 , -4084864.356,  1858337.667, -2345182.313,  -671943.116,  1188017.457],
       [ -508264.159,   815419.54 ,  4852098.577,  1363270.625, -2738422.594,  -638833.172, -1605411.823, -1539856.993],
       [  540694.265, -4084864.356,  1363270.625,  6506815.302,  -364107.898,   519514.326, -1539856.993, -2941465.272],
       [-3269537.137,  1858337.667, -2738422.594,  -364107.898,  5526999.02 , -1981120.376,   480960.711,   486890.607],
       [ 1858337.667, -2345182.313,  -638833.172,   519514.326, -1981120.376,  4824214.756,   761615.881, -2998546.769],
       [-2129668.827,  -671943.116, -1605411.823, -1539856.993,   480960.711,   761615.881,  3254119.94 ,  1450184.228],
       [ -397217.842,  1188017.457, -1539856.993, -2941465.272,   486890.607, -2998546.769,  1450184.228,  4751994.584]])

Şimdi herhangi bir eleman için $\mathbf{J}$ ve eleman sertlik matrisini hesaplayan fonksiyonumuzu tanımlayalım:

In [63]:
noktalar = np.array([[0, -0.32],
                     [0, -0.16],
                     [0,0],
                     [0.16, -0.2],
                     [0.16,-0.1],
                     [0.13, 0],
                     [0.32,-0.08],
                     [0.29,-0.04],
                     [0.26,0]])

mesh = {1:[0,3,4,1],
        2:[4,5,2,1],
        3:[4,7,8,5],
        4:[3,6,7,4]}

In [64]:
def lokal_jakobyen(mesh, noktalar, eleman_no):
    # Elemana ait noktalarin global indislerini içeren dizi
    eleman_noktalar = mesh[eleman_no]
    c_şekil = sp.Matrix([0, 0])
    
    for i in range(4):
        c_şekil[0] += x_[i] * şekil[i]
        c_şekil[1] += y_[i] * şekil[i]
        
    j = c_şekil.jacobian([xi, eta]).T
    
    for nokta, global_nokta in enumerate(eleman_noktalar):
        print(noktalar[global_nokta])
        j = j.subs(x_[nokta],noktalar[global_nokta][0]) 
        j = j.subs(y_[nokta],noktalar[global_nokta][1])

    J = sp.Matrix.zeros(4,4)
    J[:2,:2] = j.inv()
    J[2:,2:] = j.inv()
    
    return J, j


def K_e_eleman(mesh, noktalar,eleman_no):

    J_e,j_e = lokal_jakobyen(mesh, noktalar,eleman_no)

    K_ele_sym=N.T*J_e.T*H.T*C*H*J_e*N*j_e.det()

    koordinat_sayısı = 8
    K_e = np.zeros((koordinat_sayısı,koordinat_sayısı))

    xi = sp.Symbol("xi")
    eta = sp.Symbol("eta")
    
    for i in range(koordinat_sayısı):
        for j in range(koordinat_sayısı):
            geçici_fonksiyon = lambdify( (xi,eta), K_ele_sym[i,j], 'math' )
            K_e[i,j] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]
            
    return K_e

K_1 = K_e_eleman(mesh, noktalar,1) 
K_1

[ 0.   -0.32]
[ 0.16 -0.2 ]
[ 0.16 -0.1 ]
[ 0.   -0.16]


array([[ 2302296.384,   297511.034, -1430926.962,  -970523.149,    57300.589,  -952553.774,  -928670.011,  1625565.889],
       [  297511.034,  4377491.538,  -695797.875,  1498760.792,  -952553.774, -1979530.023,  1350840.614, -3896722.307],
       [-1430926.962,  -695797.875,  9899373.25 , -3666943.181, -2344428.195,  1743866.258, -6124018.093,  2618874.798],
       [ -970523.149,  1498760.792, -3666943.181,  7979729.985,  2018591.532, -5335499.216,  2618874.798, -4142991.561],
       [   57300.589,  -952553.774, -2344428.195,  2018591.532,  3718054.568,   -95514.609, -1430926.962,  -970523.149],
       [ -952553.774, -1979530.023,  1743866.258, -5335499.216,   -95514.609,  5816268.447,  -695797.875,  1498760.792],
       [ -928670.011,  1350840.614, -6124018.093,  2618874.798, -1430926.962,  -695797.875,  8483615.066, -3273917.537],
       [ 1625565.889, -3896722.307,  2618874.798, -4142991.561,  -970523.149,  1498760.792, -3273917.537,  6540953.076]])

In [65]:
# Simdi global matrisi olusturalim

np.set_printoptions(linewidth=300, precision=3)


K_global = np.zeros((18,18))
for el in range(1,5):
    K_e = K_e_eleman(mesh, noktalar,el)
    eleman_noktalar = mesh[el]
    for lokal_x, global_x in enumerate(eleman_noktalar):
        for lokal_y, global_y in enumerate(eleman_noktalar):
            K_global[2*global_x:2*global_x+2,2*global_y:2*global_y+2] += K_e[2*lokal_x:2*lokal_x+2,2*lokal_y:2*lokal_y+2]
            

K_global

[ 0.   -0.32]
[ 0.16 -0.2 ]
[ 0.16 -0.1 ]
[ 0.   -0.16]
[ 0.16 -0.1 ]
[0.13 0.  ]
[0. 0.]
[ 0.   -0.16]
[ 0.16 -0.1 ]
[ 0.29 -0.04]
[0.26 0.  ]
[0.13 0.  ]
[ 0.16 -0.2 ]
[ 0.32 -0.08]
[ 0.29 -0.04]
[ 0.16 -0.1 ]


array([[  2302296.384,    297511.034,   -928670.011,   1625565.889,         0.   ,         0.   ,  -1430926.962,   -970523.149,     57300.589,   -952553.774,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ],
       [   297511.034,   4377491.538,   1350840.614,  -3896722.307,         0.   ,         0.   ,   -695797.875,   1498760.792,   -952553.774,  -1979530.023,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ],
       [  -928670.011,   1350840.614,  11737735.006,  -1823733.309,    480960.711,    761615.881,  -6124018.093,   2618874.798,  -3560595.79 ,  -1367740.991,  -1605411.823,  -1539856.993,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ,         0.   ],
       [  1625565.889,  -3896722.307,  -1823733.309,  11292947.66 ,    486890.607,  -2998546.769,   2618874.798,  -4142991.561,  -1367740.991,   2686778.249,  -153

In [78]:
# simdi f'i olusturalim

f_global = np.array([0,0,0,0,0,-325,0,0,0,0,0,-1950,0,0,-2000,0,0,-1625])
# f_global = np.array([0,0,0,0,0,0,0,0,0,0,0,-1950,0,0,-2000,0,0,-3900])
f_global.shape

(18,)

In [67]:
# simdi B.C. leri uygulayalim - galiba gerek yok - olmadigini anlat
beta  = 1
gamma = 0.0 # yatay hiz degerleri
# h = 0.005 # kalinlik

# def dirichlet_uygula(kenar, K_global, f_global, alpha, beta, gamma, A, k):

#     for nokta in kenar:
#         K_global[nokta,nokta]+= 1*beta*k*A/alpha  
#         f_global[nokta] += 1*gamma*k*A/alpha
    
#     return K_global, f_global

# K_global, f_global = dirichlet_uygula(plaka.sol_kenar_noktalar, K_global, f_global, alpha, beta, gamma, A, k)

In [79]:
# sistem cozumu

from scipy.linalg import solve

u = solve(K_global,f_global)
# Çözüm vektörünü çözüm ağına göre yeniden boyutlandırmak
u

/tmp/ipykernel_3013/169894189.py:5: LinAlgWarning: Ill-conditioned matrix (rcond=2.38992e-18): result may not be accurate.
  u = solve(K_global,f_global)


array([ 6.869e+11,  1.060e+12,  2.517e+11,  1.060e+12, -1.835e+11,  1.060e+12,  3.605e+11,  1.495e+12,  8.850e+10,  1.495e+12, -1.835e+11,  1.413e+12,  3.410e+10,  1.930e+12, -7.471e+10,  1.848e+12, -1.835e+11,  1.767e+12])